<a href="https://colab.research.google.com/github/heyggun/bigdata_analysis_test/blob/main/DataManim_type2_Classification_%EC%84%9C%EB%B9%84%EC%8A%A4%EC%9D%B4%ED%83%88%EC%98%88%EC%B8%A1.(RF%3D0.84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형 2유형

## 분류

In [1]:
import pandas as pd
import numpy as np

### 서비스 이탈 예측 데이터

- 데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)

- x_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv

- y_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv

- x_test : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv

- x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv

- 데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형

In [2]:
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv')
x_label = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv')

In [3]:
display(x_train.head(3))
print(x_train.shape)

display(y_train.head(3))
print(y_train.shape)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15799217,Zetticci,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,15748986,Bischof,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,15722004,Hsiung,543,France,Female,31,4,138317.94,1,0,0,61843.73


(6499, 12)


,CustomerId,Exited
0,15799217,0
1,15748986,0
2,15722004,0


(6499, 2)


In [4]:
display(x_test.head(3))
print(x_test.shape)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15601012,Abdullah,802,France,Female,60,3,92887.06,1,1,0,39473.63
1,15734762,Ignatiev,602,France,Female,56,3,115895.22,3,1,0,4176.17
2,15586757,Anenechukwu,801,France,Female,32,4,75170.54,1,1,1,37898.50


(3501, 12)


In [5]:
# 결측치

print(x_train.isna().sum())
print(x_test.isna().sum())

CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64


In [6]:
X_train = x_train.drop(columns=['Surname','CustomerId'], axis=1)
X_test = x_test.drop(columns=['Surname','CustomerId'], axis=1)

In [7]:
# 범주형, 수치형 변수 파악

obj = X_train.select_dtypes(include='object').columns
num = X_train.select_dtypes(include='number').columns


print(obj)
print(num)

Index(['Geography', 'Gender'], dtype='object')
Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')


In [8]:
# 범주형 변수 전처리

from sklearn.preprocessing import LabelEncoder

Encoder = LabelEncoder()
df = pd.concat([X_train.assign(ind='train'), X_test.assign(ind='test')])
df[obj] = df[obj].apply(Encoder.fit_transform)

In [9]:
X_train_l = df[df['ind']=='train'].drop(columns='ind', axis=1)
X_test_l = df[df['ind']=='test'].drop(columns='ind', axis=1)

In [10]:
# 수치형 변수 전처리

from sklearn.preprocessing import StandardScaler

Scaler = StandardScaler()

X_train_s = Scaler.fit_transform(X_train_l)
X_test_s = Scaler.transform(X_test_l)

In [11]:
# 모델링을 위한 data split

from sklearn.model_selection import train_test_split

X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_train_s, y_train['Exited'],
                                                            test_size=0.3,
                                                            random_state=1234)
                                                            

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

(4549, 10)
(1950, 10)
(4549,)
(1950,)


In [12]:
# 모델링

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(X_train_m, y_train_m)
pred = model.predict_proba(X_test_m)[:,1]
pred

array([0.9 , 0.08, 0.04, ..., 0.05, 0.02, 0.03])

In [13]:
# 평가

from sklearn.metrics import roc_auc_score

print(roc_auc_score(y_test_m, pred))

0.8479452983314072


In [14]:

model.fit(X_train_l, y_train['Exited'])
prediction = model.predict_proba(X_test_l)[:,1]
prediction_label = model.predict(X_test_l)

result = pd.DataFrame({'CustomerId':x_test['CustomerId'], 'Exited':prediction_label})
result

,CustomerId,Exited
0,15601012,1
1,15734762,1
2,15586757,0
3,15590888,0
4,15726087,0
...,...,...
3496,15733966,1
3497,15669994,0
3498,15712403,1
3499,15643819,0


In [15]:
print(roc_auc_score(y_test[['Exited']], prediction))

0.848585955437147
